<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [28]:
# avoid decoding problems
df = pd.read_csv("train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

## Split the data into train test

In [29]:
y_true = df['is_duplicate']
df = df.drop('is_duplicate', axis=1)
print(len(y_true))
df.head()

404290


,id,qid1,qid2,question1,question2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(df, y_true, stratify=y_true, test_size=0.3, random_state = 22)

In [2]:
print(X_train.shape)
print(X_test.shape)


NameError: name 'X_train' is not defined

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions_train = list(X_train['question1']) + list(X_train['question2'])
questions_test = list(X_test['question1']) + list(X_test['question2'])

tfidf = TfidfVectorizer(lowercase=False,)
tfidf.fit(list(X_train['question1']) + list(X_train['question2']))
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
# X_train_tfidf_q1 = tfidf.transform(list(X_train['question1']))
# X_train_tfidf_q2 = tfidf.transform(list(X_train['question2']))

# X_test_tfidf_q1 = tfidf.transform(list(X_test['question1']))
# X_test_tfidf_q2 = tfidf.transform(list(X_test['question2']))

NameError: name 'X_train' is not defined

In [0]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# # merge texts
# questions = list(df['question1']) + list(df['question2'])

# tfidf = TfidfVectorizer(lowercase=False, )
# tfidf.fit_transform(questions)

# # dict key:word and value:tf-idf score
# word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [36]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('D:\Installation\Anaconda\Lib\site-packages\en_core_web_sm2\en_core_web_sm-2.1.0')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_train['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
X_train['q1_feats_m'] = list(vecs1)

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_test['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
X_test['q1_feats_m'] = list(vecs1)


100%|██████████████████████████████████████████████████████████████████████████| 121287/121287 [22:11<00:00, 82.43it/s]


In [35]:
nlp = spacy.load('D:\Installation\Anaconda\Lib\site-packages\en_core_web_sm2\en_core_web_sm-2.1.0')

vecs2 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu2 in tqdm(list(X_train['question2'])):
    doc2 = nlp(qu2) 
    # 384 is the number of dimensions of vectors 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
X_train['q2_feats_m'] = list(vecs2)

vecs2 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu2 in tqdm(list(X_test['question2'])):
    doc2 = nlp(qu2) 
    # 384 is the number of dimensions of vectors 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
X_test['q2_feats_m'] = list(vecs2)


100%|██████████████████████████████████████████████████████████████████████████| 121287/121287 [22:44<00:00, 88.89it/s]


In [37]:
import pickle
if os.path.isfile("x_train_w2v.pickle"):
    print("File is present loading variable")
    pickle_in = open("x_train_w2v.pickle","rb")
    X_train = pickle.load(pickle_in)
    print("Completed!!!")
else:
    pkl_in = open("x_train_w2v.pickle", "wb")
    pickle.dump(X_train, pkl_in)
    pkl_in.close()

In [38]:
if os.path.isfile("x_test_w2v.pickle"):
    print("File is present loading variable")
    pickle_in = open("x_test_w2v.pickle","rb")
    X_test = pickle.load(pickle_in)
    print("Completed!!!")
else:
    pkl_in = open("x_test_w2v.pickle", "wb")
    pickle.dump(X_test, pkl_in)
    pkl_in.close()

In [39]:
if os.path.isfile("y_train.pickle"):
    print("File is present loading variable")
    pickle_in = open("y_train.pickle","rb")
    y_train = pickle.load(pickle_in)
    print("Completed!!!")
else:
    pkl_in = open("y_train.pickle", "wb")
    pickle.dump(y_train, pkl_in)
    pkl_in.close()

In [40]:
if os.path.isfile("y_test2.pickle"):
    print("File is present loading variable")
    pickle_in = open("y_test2.pickle","rb")
    y_test = pickle.load(pickle_in)
    print("Completed!!!")
else:
    pkl_in = open("y_test2.pickle", "wb")
    pickle.dump(y_test, pkl_in)
    pkl_in.close()

In [41]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [44]:
X_train_df1 = X_train_dfnlp.drop(['is_duplicate'],axis=1)
X_train_df2 = X_train_dfppro.drop(['is_duplicate'],axis=1)
# X_train_df3 = X_train.drop(['qid1','qid2','question1','question2'],axis=1)
X_train_df3_q1 = pd.DataFrame(X_train.q1_feats_m.values.tolist(), index= X_train.index)
X_train_df3_q2 = pd.DataFrame(X_train.q2_feats_m.values.tolist(), index= X_train.index)

In [45]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [46]:
X_test_df1 = X_test_dfnlp.drop(['is_duplicate'],axis=1)
X_test_df2 = X_test_dfppro.drop(['is_duplicate'],axis=1)
# X_test_df3 = X_test.drop(['qid1','qid2','question1','question2'],axis=1)
X_test_df3_q1 = pd.DataFrame(X_test.q1_feats_m.values.tolist(), index= X_test.index)
X_test_df3_q2 = pd.DataFrame(X_test.q2_feats_m.values.tolist(), index= X_test.index)

In [53]:
X_test_df3_q1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
27658,14.922910,-53.948082,-37.864075,-119.962747,14.833321,22.331665,28.771530,-10.901183,75.696638,125.669407,...,25.257654,-16.848488,27.148519,23.782143,-56.321126,92.728355,-36.377258,-13.344163,49.023959,33.849448
185452,78.744826,-37.219667,-72.192692,-24.898964,-2.305341,32.453925,76.251791,63.623405,-43.570591,64.972474,...,43.537686,18.614640,52.566494,25.942430,-3.298308,19.982710,-41.218373,26.162981,-15.983819,0.324986


In [47]:
X_train.head(1)

,id,qid1,qid2,question1,question2,q2_feats_m,q1_feats_m
349310,349310,30295,477949,What are some of the most painful and/or cruel...,Who is known to be the most cruel person in wo...,"[-39.74403768777847, -52.204767391085625, -119...","[-10.54879641532898, -51.50204508379102, -100...."


In [48]:
X_train_dfnlp.head(1)

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
349310,349310,0,0.499988,0.399992,0.428565,0.299997,0.416663,0.333331,1.0,0.0,3.0,13.5,62,55,59,57,0.169492


In [49]:
X_train_dfppro.head(1)

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
349310,349310,0,0.499988,0.399992,0.428565,0.299997,0.416663,0.333331,1.0,0.0,3.0,13.5,62,55,59,57,0.169492


In [50]:
X_test.head(1)

,id,qid1,qid2,question1,question2,q2_feats_m,q1_feats_m
27658,27658,7338,6103,What is the quickest way to get meth out your ...,How do I get meth out of my system from smokin...,"[-24.486527264118195, -96.80893194675446, -71....","[14.922909617424011, -53.94808179140091, -37.8..."


In [12]:
# df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
# df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# X_train_dfnlp,X_test_dfnlp, y_train_dfnlp, y_test_dfnlp = train_test_split(df1, y_true, stratify=y_true, test_size=0.3, random_state = 22)
# X_train_dfppro,X_test_dfppro, y_train_dfppro, y_test_dfppro = train_test_split(df1, y_true, stratify=y_true, test_size=0.3, random_state = 22)

In [51]:
print("Number of features in nlp dataframe :", X_train_dfnlp.shape)
print("Number of features in preprocessed dataframe :", X_train_dfppro.shape)
print("Number of features in question1 tfidf  dataframe :", X_train.shape)
print("Number of features in question2 tfidf  dataframe :", X_train.shape)
# print("Number of features in final dataframe  :", X_train_dfnlp.shape[1]+X_train_dfppro.shape[1]+X_train_tfidf_q1_5k.shape[1]+X_train_tfidf_q2_5k.shape[1])

Number of features in nlp dataframe : (283003, 17)
Number of features in preprocessed dataframe : (283003, 17)
Number of features in question1 tfidf  dataframe : (283003, 7)
Number of features in question2 tfidf  dataframe : (283003, 7)


In [14]:
X_train.head()

,id,qid1,qid2,question1,question2,q1_feats_m,q2_feats_m
349310,349310,30295,477949,What are some of the most painful and/or cruel...,Who is known to be the most cruel person in wo...,"[-10.54879641532898, -51.50204508379102, -100....","[-39.74403768777847, -52.204767391085625, -119..."
247517,247517,360767,360768,What do you do when you're wrongly accused by ...,Have you been wrongly accused by a girl or a w...,"[-27.63896381855011, -2.176826000213623, -109....","[-5.970794200897217, 14.43298014998436, -149.4..."
320565,320565,324632,446136,What temperature shower should I take when I h...,Can we take steam while having fever?,"[132.71159029006958, -87.76625883579254, 91.41...","[22.90628170967102, -4.5829997062683105, -25.3..."
379836,379836,511386,511387,What are the cost drivers for business insuran...,What are the cost drivers for legal fees for a...,"[209.55288910865784, 39.27708327770233, -92.49...","[237.73450660705566, 63.93543916940689, -182.3..."
325603,325603,451864,451865,Why don't you ever see ghost dogs?,Can dogs see ghosts?,"[38.74923479557037, 26.065393924713135, -83.75...","[80.86676919460297, 79.11699056625366, -29.870..."


In [0]:
# df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
# df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
# df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [0]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [0]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [0]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,121.929927,100.083900,72.497894,115.641800,-48.370870,34.619058,-172.057787,-92.502617,113.223315,50.562441,...,12.397642,40.909519,8.150261,-15.170692,18.007709,6.166999,-30.124163,3.700902,-1.757693,-1.818058
1,-78.070939,54.843781,82.738482,98.191872,-51.234859,55.013510,-39.140730,-82.692352,45.161489,-9.556289,...,-21.987077,-12.389279,20.667979,2.202714,-17.142454,-5.880972,-10.123963,-4.890663,-13.018389,-5.219310
2,-5.355015,73.671810,14.376365,104.130241,1.433537,35.229116,-148.519385,-97.124595,41.972195,50.948731,...,3.027700,14.025767,-2.960312,-3.206544,4.355141,2.936152,-20.199555,9.816351,11.894366,-8.798819
3,5.778359,-34.712038,48.999631,59.699204,40.661263,-41.658731,-36.808594,24.170655,0.235600,-29.407290,...,13.100007,1.405670,-1.891076,-7.882638,18.000561,12.106918,-10.507835,5.243834,10.158340,5.886351
4,51.138220,38.587312,123.639488,53.333041,-47.062739,37.356212,-298.722753,-106.421119,106.248914,65.880707,...,13.906532,43.461721,11.519207,-22.468284,45.431128,8.161224,-35.373910,7.728865,9.592849,5.447336


In [0]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,125.983301,95.636485,42.114702,95.449980,-37.386295,39.400078,-148.116070,-87.851475,110.371966,62.272814,...,16.165592,33.030668,7.019996,-14.793959,15.437511,8.199658,-25.070834,1.571619,1.603738,0.305645
1,-106.871904,80.290331,79.066297,59.302092,-42.175328,117.616655,-144.364237,-127.131513,22.962533,25.397575,...,-4.901128,-4.565393,41.520751,-0.727564,-16.413776,-7.373778,2.638877,-7.403457,2.703070,0.408040
2,7.072875,15.513378,1.846914,85.937583,-33.808811,94.702337,-122.256856,-114.009530,53.922293,60.131814,...,8.359966,-2.165985,10.936580,-16.531660,14.681230,15.633759,-1.210901,14.183826,11.703135,10.148075
3,39.421531,44.136989,-24.010929,85.265863,-0.339022,-9.323137,-60.499651,-37.044763,49.407848,-23.350150,...,3.311411,3.788879,13.398598,-6.592596,6.437365,5.993293,2.732392,-3.727647,5.614115,6.023693
4,31.950101,62.854106,1.778164,36.218768,-45.130875,66.674880,-106.342341,-22.901008,59.835938,62.663961,...,-2.403870,11.991204,8.088483,-15.090201,8.375166,1.727225,-6.601129,11.317413,11.544603,2.478689


In [0]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 384
Number of features in question2 w2v  dataframe : 384
Number of features in final dataframe  : 794


In [54]:
# storing the final features to csv file
if not os.path.isfile('X_train_final_features_w2v.csv'):
    X_train_df3_q1['id']=X_train_df1['id']
    X_train_df3_q2['id']=X_train_df1['id']
    X_train_df1  = X_train_df1.merge(X_train_df2, on='id',how='left')
    X_train_df2  = X_train_df3_q1.merge(X_train_df3_q2, on='id',how='left')
    result  = X_train_df1.merge(X_train_df2, on='id',how='left')
    result.to_csv('X_train_final_features_w2v.csv')

In [55]:
result.head(2)

,id,cwc_min_x,cwc_max_x,csc_min_x,csc_max_x,ctc_min_x,ctc_max_x,last_word_eq_x,first_word_eq_x,abs_len_diff_x,...,86_y,87_y,88_y,89_y,90_y,91_y,92_y,93_y,94_y,95_y
0,349310,0.499988,0.399992,0.428565,0.299997,0.416663,0.333331,1.0,0.0,3.0,...,81.606348,17.081403,31.487341,9.267629,36.68113,5.642915,-26.998967,-2.774215,-68.742285,28.984116
1,247517,0.999975,0.799984,0.571420,0.499994,0.615380,0.571424,1.0,0.0,1.0,...,112.049004,-108.436734,-58.761662,-29.753687,-69.11599,41.314589,36.405801,56.572525,-94.981913,46.277536


In [56]:
# storing the final features to csv file
if not os.path.isfile('X_test_final_features_w2v.csv'):
    X_test_df3_q1['id']=X_test_df1['id']
    X_test_df3_q2['id']=X_test_df1['id']
    X_test_df1  = X_test_df1.merge(X_test_df2, on='id',how='left')
    X_test_df2  = X_test_df3_q1.merge(X_test_df3_q2, on='id',how='left')
    result  = X_test_df1.merge(X_test_df2, on='id',how='left')
    result.to_csv('X_test_final_features_w2v.csv')

In [57]:
result.head(2)

,id,cwc_min_x,cwc_max_x,csc_min_x,csc_max_x,ctc_min_x,ctc_max_x,last_word_eq_x,first_word_eq_x,abs_len_diff_x,...,86_y,87_y,88_y,89_y,90_y,91_y,92_y,93_y,94_y,95_y
0,27658,0.599988,0.599988,0.166664,0.124998,0.363633,0.307690,0.0,0.0,2.0,...,120.818906,8.346355,25.471466,133.091496,34.817693,65.740093,-36.250895,102.048714,-55.348558,40.101771
1,185452,0.249994,0.199996,0.714276,0.714276,0.545450,0.499996,0.0,1.0,1.0,...,98.929229,-12.260782,-9.783287,91.931960,13.154943,103.950557,35.657100,52.941494,29.964000,-2.704172
